In [25]:
# NAME: HAARISH V
# REG NO: 212223230067
# SIMPLE CHAIN
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "one piece"})

'One Piece is a popular Japanese manga and anime series created by Eiichiro Oda. The story follows Monkey D. Luffy, a young pirate with the ability to stretch his body like rubber after eating a Devil Fruit. Luffy sets out on a journey to find the legendary treasure known as the One Piece and become the Pirate King.\n\nThroughout his journey, Luffy gathers a diverse crew of pirates, known as the Straw Hat Pirates, and they embark on various adventures while facing powerful enemies and uncovering mysteries from the past. The series is known for its intricate world-building, well-developed characters, and epic battles.\n\nOne Piece has been serialized in Weekly Shonen Jump magazine since 1997 and has spawned a successful anime adaptation, as well as numerous films, video games, and merchandise. It is one of the best-selling manga series of all time and has a massive global fanbase.'

In [12]:
# COMPLEX CHAIN
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch   

vectorstore = DocArrayInMemorySearch.from_texts(
    ["Generative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.","(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently."],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("what is generative ai?")
retriever.get_relevant_documents("what is the full form of LCEL")

[Document(page_content='(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently.'),
 Document(page_content='Generative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.')]

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [14]:
chain.invoke({"question": "what is the full form of LCEL?"})

'LangChain Expression Language'

In [19]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [20]:
inputs.invoke({"question": "what is the full form of LCEL?"})

{'context': [Document(page_content='(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently.'),
  Document(page_content='Generative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.')],
 'question': 'what is the full form of LCEL?'}